In [338]:
import glob

#IMPORTING FUNCTIONS FROM ./defs
from importlib.machinery import SourceFileLoader

In [342]:
path = glob.glob("./database_raw/Dataset A/zzzA*.txt", recursive=True)

31

In [343]:
def_initDatabase = SourceFileLoader("initDatabase", "defs/initDatabase.py").load_module()
df = def_initDatabase.initDatabase(path)

df

,Time,Real,Real,Real,Real,Real,Real,Real,Real,Real,...,Real,Real,Real,Real,Real,Real,Real,Real,Real,Scenario
0,0.0000000E+00,0.004954129,-0.00135893,0.004767455,-0.008526795,-0.00118725,0.004382603,-0.0006517582,-0.03157736,-0.007815173,...,-0.001459152,0.002673313,-0.001022762,0.0008713845,-0.003662355,0.005480237,-0.004953712,0.003154866,-0.002783455,1
1,+9.7656250E-04,-0.002692187,0.01322381,-0.0121593,0.006356075,-0.01056883,0.005802823,-0.01034775,0.05124363,-0.0152378,...,-3.857538E-05,-0.002361529,0.001402684,0.0005187672,-0.003883243,-0.001152333,0.009067893,-0.002660438,-0.0009352416,1
2,+1.9531250E-03,0.01044003,-0.01754753,-0.002408652,-0.02368012,0.003805429,-0.01430519,0.02805205,-0.001950361,0.02293378,...,-0.001821447,0.00918106,-0.0009873509,0.004117209,-0.005031459,0.00692381,0.008965768,0.006289423,-0.002047138,1
3,+2.9296880E-03,-0.01978657,0.005272564,-0.003470697,0.004499115,-0.02361103,0.004458822,0.0128046,0.02860562,0.009632573,...,0.007826921,0.0005375072,0.009287003,0.0009200573,-0.001097824,-0.00259589,0.01336877,-0.00136384,0.003096842,1
4,+3.9062500E-03,-0.003795579,0.006210797,0.0007152557,-2.169609E-05,-0.008823663,0.01714509,0.01271839,0.04208693,0.01051033,...,-0.005140658,0.01108169,-0.003524981,0.01101013,-0.005373763,0.007328894,-0.007781997,0.008711606,-0.0002473295,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8126459,+2.5599510E+02,0.02545519,-0.03249098,0.1050803,-0.1540732,-0.04625436,-0.03826273,0.07049614,-0.4399057,-0.02013387,...,0.1244391,0.1128331,0.2221611,-0.03146146,0.02900424,0.01728192,-0.2628244,-0.02576224,0.1617818,0
8126460,+2.5599610E+02,-0.1574922,-0.1324157,0.1176808,-0.07484005,-0.18706,-0.01591686,0.03650741,0.1451141,-0.1674283,...,0.2874352,0.1402219,0.06985807,0.06060453,-0.1466551,-0.2539414,-0.0247058,0.0001273528,-0.07298092,0
8126461,+2.5599710E+02,-0.03476283,-0.0293199,-0.1597718,0.167345,0.0222498,-0.02767406,-0.02683485,0.5293479,0.1675857,...,0.1460954,0.1425573,-0.164077,0.01439276,0.0982991,-0.125229,-0.1318343,0.04336017,-0.08080912,0
8126462,+2.5599800E+02,0.0124216,-0.2350311,-0.320868,-0.4181285,0.1302845,-0.05136404,0.2770998,0.03592081,0.1339608,...,-0.08324742,-0.01638616,0.2248302,-0.3032672,-0.2055517,0.1159847,0.1007911,-0.1339332,0.07721111,0
